<a href="https://colab.research.google.com/github/sqbitegh/Colabs/blob/main/TPU_Whisper_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tune Whisper on TPU

We can verify that we've been assigned a GPU and view its specifications:

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


We'll employ several popular Python packages to fine-tune the Whisper model.
We'll use `datasets[audio]` to download and prepare our training data, alongside
`transformers` and `accelerate` to load and train our Whisper model.
We'll also require the `soundfile` package to pre-process audio files,
`evaluate` and `jiwer` to assess the performance of our model, and
`tensorboard` to log our metrics. Finally, we'll use `gradio` to build a
flashy demo of our fine-tuned model.

## Load Dataset

In [1]:
!pip install --upgrade --quiet datasets[audio] transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from transformers import WhisperFeatureExtractor, WhisperTokenizer
obsoletefeature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny")
obsoletetokenizer = WhisperTokenizer.from_pretrained("openai/whisper-tiny", language="Polish", task="transcribe")


tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [4]:
#cached dataset loading
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

dataset_in_hub_path = "ambind/preprocessed_commonvoice_pl"
dataset_ready = False
try:
        raise FileNotFoundError #needed to reload from source repo as otherwise hfhub could provide cached content
        data_files = {"train": f"{dataset_in_hub_path}/train*.parquet", "test": f"{dataset_in_hub_path}/test*.parquet"}
        common_voice["train"] = load_dataset(path="ambind/preprocessed_commonvoice_pl", split ="train")
        common_voice["test"] = load_dataset(path="ambind/preprocessed_commonvoice_pl", split ="test")
        dataset_ready = True
       # print("dataset loaded from " + dataset_in_hub_path + " content " + common_voice["train"])
except FileNotFoundError:
        print(f"Dataset '{dataset_in_hub_path}' not found on the Hugging Face Hub.")
except Exception as e:
        print(f"An unexpected error occurred: {e}")
if not dataset_ready:
  common_voice_source = DatasetDict()


  #debug
  common_voice_source["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "pl", split="train[:2%]+validation[:2%]")
  common_voice_source["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "pl", split="test[:2%]")
  #common_voice["validation"] = load_dataset("mozilla-foundation/common_voice_11_0", "pl", split="validation[:2%]")

  #full
  #common_voice_source["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "pl",split="train+validation")
  #common_voice_source["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "pl", split="test")

  print(common_voice_source)



Dataset 'ambind/preprocessed_commonvoice_pl' not found on the Hugging Face Hub.


README.md:   0%|          | 0.00/14.4k [00:00<?, ?B/s]

common_voice_11_0.py:   0%|          | 0.00/8.13k [00:00<?, ?B/s]

languages.py:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

release_stats.py:   0%|          | 0.00/60.9k [00:00<?, ?B/s]

...

Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 4686it [00:00, 170579.48it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 6136it [00:00, 155702.67it/s]


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 497
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 166
    })
})


Pre process data in case its loaded from source dataset repository, otherwise skip

In [5]:
from datasets import Audio


common_voice_source = common_voice_source.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])
print(common_voice_source["train"][0])

common_voice_source = common_voice_source.cast_column("audio", Audio(sampling_rate=16000))
print(common_voice_source["train"][0])



{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/c35bb1e1b0a4cbcb6a811735439fdd61936c6f426fc9c1e20a2f0c00a00daf08/pl_train_0/common_voice_pl_20990285.mp3', 'array': array([ 0.00000000e+00,  3.58114180e-15, -4.51483935e-15, ...,
        2.94640543e-08, -1.09114431e-07, -3.94480764e-07]), 'sampling_rate': 48000}, 'sentence': 'Nie wiem właściwie, dlaczego tego nie zrobili.'}
{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/c35bb1e1b0a4cbcb6a811735439fdd61936c6f426fc9c1e20a2f0c00a00daf08/pl_train_0/common_voice_pl_20990285.mp3', 'array': array([ 1.81898940e-12, -2.27373675e-12, -2.72848411e-12, ...,
       -5.09688107e-07, -3.80561687e-07, -1.59896445e-07]), 'sampling_rate': 16000}, 'sentence': 'Nie wiem właściwie, dlaczego tego nie zrobili.'}


In [6]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]
    # compute log-Mel input features from input audio array
    batch["input_features"] = obsoletefeature_extractor(audio["array"], sampling_rate=16000).input_features[0]
    #batch["input_features"] .input_features[0]
    # encode target text to label ids

    batch["labels"] = obsoletetokenizer(batch["sentence"]).input_ids
    return batch


print(obsoletefeature_extractor)
t0 = common_voice_source["train"][0]
print(t0)
au = t0["audio"]
print(au)
auarr = au["array"]
print(auarr)
x = obsoletefeature_extractor(auarr, sampling_rate=16000)
print(x)

common_voice = common_voice_source.map(prepare_dataset, remove_columns=common_voice_source.column_names["train"], num_proc=1).with_format("torch")
print(common_voice)

WhisperFeatureExtractor {
  "chunk_length": 30,
  "feature_extractor_type": "WhisperFeatureExtractor",
  "feature_size": 80,
  "hop_length": 160,
  "n_fft": 400,
  "n_samples": 480000,
  "nb_max_frames": 3000,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "WhisperProcessor",
  "return_attention_mask": false,
  "sampling_rate": 16000
}

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/c35bb1e1b0a4cbcb6a811735439fdd61936c6f426fc9c1e20a2f0c00a00daf08/pl_train_0/common_voice_pl_20990285.mp3', 'array': array([ 1.81898940e-12, -2.27373675e-12, -2.72848411e-12, ...,
       -5.09688107e-07, -3.80561687e-07, -1.59896445e-07]), 'sampling_rate': 16000}, 'sentence': 'Nie wiem właściwie, dlaczego tego nie zrobili.'}
{'path': '/root/.cache/huggingface/datasets/downloads/extracted/c35bb1e1b0a4cbcb6a811735439fdd61936c6f426fc9c1e20a2f0c00a00daf08/pl_train_0/common_voice_pl_20990285.mp3', 'array': array([ 1.81898940e-12, -2.27373675e-12, -2.72848411e-12

Map:   0%|          | 0/497 [00:00<?, ? examples/s]

Map:   0%|          | 0/166 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 497
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 166
    })
})


### Load WhisperTokenizer, WhisperProcessor, WhisperFeatureExtractor and model

In [1]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg

Repository: 'deb https://ppa.launchpadcontent.net/jonathonf/ffmpeg-4/ubuntu/ jammy main'
Description:
Backport of FFmpeg 4 and associated libraries. Now includes AOM/AV1 support!

FDK AAC is not compatible with GPL and FFmpeg can't be redistributed with it included. Please don't ask for it to be added to this public PPA.

---

PPA supporters:

BigBlueButton (https://bigbluebutton.org)

---

Donate to FFMPEG: https://ffmpeg.org/donations.html
Donate to Debian: https://www.debian.org/donations
Donate to this PPA: https://ko-fi.com/jonathonf
More info: https://launchpad.net/~jonathonf/+archive/ubuntu/ffmpeg-4
Adding repository.
Adding deb entry to /etc/apt/sources.list.d/jonathonf-ubuntu-ffmpeg-4-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/jonathonf-ubuntu-ffmpeg-4-jammy.list
Adding key to /etc/apt/trusted.gpg.d/jonathonf-ubuntu-ffmpeg-4.gpg with fingerprint 4AB0F789CBA31744CC7DA76A8CF63AD3F06FC659
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [1

In [2]:

#!pip install --quiet "transformers==4.28.1" "datasets[audio]" "accelerate==0.19.0"
!pip install --quiet "transformers==4.28.1" "datasets[audio]" "accelerate==0.19.0" "torchaudio"
!pip uninstall -y tensorflow && pip install "tensorflow-cpu==2.15.0"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 MB 4.9 MB/s eta 0:00:00


In [3]:
import transformers
import accelerate

print(f"Transformers version: {transformers.__version__}")
print(f"Accelerate version: {accelerate.__version__}")

Transformers version: 4.28.1
Accelerate version: 0.19.0


In [4]:
# Select CUDA device index
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor

from transformers import WhisperForConditionalGeneration

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny")
#model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny", load_in_8bit=False, device_map="auto")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

In [5]:
from transformers import WhisperForConditionalGeneration
from transformers import WhisperTokenizer
from transformers import WhisperFeatureExtractor
from transformers import WhisperProcessor


firstrun = True
try:
    raise EnvironmentError
    model = WhisperForConditionalGeneration.from_pretrained("ambind/whisper-tiny-pl")
    firstrun = False
    print("Model loaded successfully from!")
except (OSError, ValueError, EnvironmentError) as e:
    print("No cached model found, instantiate source model.")
    #model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny", load_in_8bit=False, device_map="auto")
    model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
    firstrun = True

processor = WhisperProcessor.from_pretrained("openai/whisper-tiny", language="Polish", task="transcribe")
#feature_extractor = processor.feature_extractor
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny")
tokenizer = processor.tokenizer

#old way:


No cached model found, instantiate source model.


tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [6]:
from transformers import Seq2SeqTrainer



In [7]:
#cached dataset loading
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

dataset_in_hub_path = "ambind/preprocessed_commonvoice_pl"
dataset_ready = False
try:
        #raise FileNotFoundError #needed to reload from source repo as otherwise hfhub could provide cached content
        data_files = {"train": f"{dataset_in_hub_path}/train*.parquet", "test": f"{dataset_in_hub_path}/test*.parquet"}
        common_voice["train"] = load_dataset(path="ambind/preprocessed_commonvoice_pl", split ="train")
        common_voice["test"] = load_dataset(path="ambind/preprocessed_commonvoice_pl", split ="test")
        dataset_ready = True
       # print("dataset loaded from " + dataset_in_hub_path + " content " + common_voice["train"])
except FileNotFoundError:
        print(f"Dataset '{dataset_in_hub_path}' not found on the Hugging Face Hub.")
except Exception as e:
        print(f"An unexpected error occurred: {e}")
if not dataset_ready:
  common_voice_source = DatasetDict()
  #debug
  #common_voice_source["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "pl", split="train[:2%]+validation[:2%]")
  #common_voice_source["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "pl", split="test[:2%]")
  #common_voice["validation"] = load_dataset("mozilla-foundation/common_voice_11_0", "pl", split="validation[:2%]")
  #full
  common_voice_source["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "pl",split="train+validation")
  common_voice_source["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "pl", split="test")
  print(common_voice_source)



README.md:   0%|          | 0.00/455 [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

train-00000-of-00048.parquet:   0%|          | 0.00/80.4M [00:00<?, ?B/s]

train-00001-of-00048.parquet:   0%|          | 0.00/86.7M [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/24833 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8294 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

We strongly advise you to upload model checkpoints directly the [Hugging Face Hub](https://huggingface.co/)

Linking the notebook to the Hub is straightforward - it simply requires entering your
Hub authentication token when prompted. Find your Hub authentication token [here](https://huggingface.co/settings/tokens):

In [7]:
from huggingface_hub import notebook_login

notebook_login()

## Training and Evaluation

In [8]:
model.generation_config.language = "polish"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

### Define a Data Collator

The data collator for a sequence-to-sequence speech model is unique in the sense that it
treats the `input_features` and `labels` independently: the  `input_features` must be
handled by the feature extractor and the `labels` by the tokenizer.

The `input_features` are already padded to 30s and converted to a log-Mel spectrogram
of fixed dimension by action of the feature extractor, so all we have to do is convert the `input_features`
to batched PyTorch tensors. We do this using the feature extractor's `.pad` method with `return_tensors=pt`.

The `labels` on the other hand are un-padded. We first pad the sequences
to the maximum length in the batch using the tokenizer's `.pad` method. The padding tokens
are then replaced by `-100` so that these tokens are **not** taken into account when
computing the loss. We then cut the BOS token from the start of the label sequence as we
append it later during training.

We can leverage the `WhisperProcessor` we defined earlier to perform both the
feature extractor and the tokenizer operations:

In [9]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch



Let's initialise the data collator we've just defined:

In [10]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

### Evaluation Metrics

We'll use the word error rate (WER) metric, the 'de-facto' metric for assessing
ASR systems. For more information, refer to the WER [docs](https://huggingface.co/metrics/wer). We'll load the WER metric from 🤗 Evaluate:

In [12]:
!pip install evaluate


ERROR: Operation cancelled by user


In [11]:
import evaluate

metric = evaluate.load("wer")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553 in run_code        │
│                                                                                                  │
│   3550 │   │   │   │   elif async_ :                                                             │
│   3551 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)               │
│   3552 │   │   │   │   else:                                                                     │
│ ❱ 3553 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                     │
│   3554 │   │   │   finally:                                                                      │
│   3555 │   │   │   │   # Reset our crash handler in place                                        │
│   3556 │   │   │   │   sys.excepthook = old_excepthook                                           │
│ in <cell line: 1>:1                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModuleNotFoundError: No module named 'evaluate'

We then simply have to define a function that takes our model
predictions and returns the WER metric. This function, called
`compute_metrics`, first replaces `-100` with the `pad_token_id`
in the `label_ids` (undoing the step we applied in the
data collator to ignore padded tokens correctly in the loss).
It then decodes the predicted and label ids to strings. Finally,
it computes the WER between the predictions and reference labels:

In [13]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    #wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    wer = 1000 #TODO add real metric

    return {"wer": wer}

### Define the Training Configuration

In the final step, we define all the parameters related to training. For more detail on the training arguments, refer to the Seq2SeqTrainingArguments [docs](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments).

In [46]:
#import os
#os.environ["XLA_USE_BF16"] = "1"

from transformers import Seq2SeqTrainingArguments


training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-tiny-pl-tpu",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-3, #debug suggested: 3.75e-5
    warmup_steps = 5 if firstrun else 2, #debug  warmup_steps=50,
    max_steps=15, #debug max_steps=400,


    evaluation_strategy="epoch",
    num_train_epochs=3,

    per_device_eval_batch_size=8,

    generation_max_length=128,

    logging_steps=100,

    metric_for_best_model="wer",
    greater_is_better=False,

    dataloader_num_workers=0,
    #tpu_num_cores=8
)

**Note**: if one does not want to upload the model checkpoints to the Hub,
set `push_to_hub=False`.

We can forward the training arguments to the 🤗 Trainer along with our model,
dataset, data collator and `compute_metrics` function:

In [47]:
from transformers import Seq2SeqTrainer

from datasets import Dataset

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"], #eval_dataset=Dataset.from_dict(common_voice["test"][:100]),  # #debug
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

We'll save the processor object once before starting training. Since the processor is not trainable, it won't change over the course of training:

In [16]:
processor.save_pretrained(training_args.output_dir)

### Training

In [48]:
trainer.train() # trainer.train(resume_from_checkpoint = True)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1662 in train                    │
│                                                                                                  │
│   1659 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1660 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1661 │   │   )                                                                                 │
│ ❱ 1662 │   │   return inner_training_loop(                                                       │
│   1663 │   │   │   args=args,                                                                    │
│   1664 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1665 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1881 in _inner_training_loop     │
│                                                                                                  │
│   1878 │   │   │   │   self._past = None                                                         │
│   1879 │   │   │                                                                                 │
│   1880 │   │   │   steps_in_epoch = (                                                            │
│ ❱ 1881 │   │   │   │   len(epoch_iterator)                                                       │
│   1882 │   │   │   │   if len_dataloader is not None                                             │
│   1883 │   │   │   │   else args.max_steps * args.gradient_accumulation_steps                    │
│   1884 │   │   │   )                                                                             │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch_xla/distributed/parallel_loader.py:36 in __len__   │
│                                                                                                  │
│    33 │   return self.next()                                                                     │
│    34                                                                                            │
│    35   def __len__(self):                                                                       │
│ ❱  36 │   return self._loader.per_device_samples()                                               │
│    37                                                                                            │
│    38   def next(self):                                                                          │
│    39 │   if xp.get_tracer_marked_step():                                                        │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch_xla/distributed/parallel_loader.py:134 in          │
│ per_device_samples                                                                               │
│                                                                                                  │
│   131 │   return PerDeviceLoader(self, torch.device(device))                                     │
│   132                                                                                            │
│   133   def per_device_samples(self):                                                            │
│ ❱ 134 │   return len(self._loader) // len(self._devices)   

Our best WER is 32.0% - not bad for 8h of training data! We can make our model more accessible on the Hub with appropriate tags and README information.
You can change these values to match your dataset, language and model
name accordingly:

In [ ]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: pl, split: test",
    "language": "pl",
    "model_name": "Whisper tiny pl",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-tiny",
    "tasks": "automatic-speech-recognition",
}

The training results can now be uploaded to the Hub. To do so, execute the `push_to_hub` command and save the preprocessor object we created:

In [ ]:
trainer.push_to_hub(**kwargs)

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

events.out.tfevents.1730811993.38c9da7b177a.1852.2:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/ambind/whisper-tiny-pl/commit/61c35f082f66edd051eb1c878d9cfb6563a66b7f', commit_message='End of training', commit_description='', oid='61c35f082f66edd051eb1c878d9cfb6563a66b7f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ambind/whisper-tiny-pl', endpoint='https://huggingface.co', repo_type='model', repo_id='ambind/whisper-tiny-pl'), pr_revision=None, pr_num=None)

## Building a Demo

Now that we've fine-tuned our model we can build a demo to show
off its ASR capabilities! We'll make use of 🤗 Transformers
`pipeline`, which will take care of the entire ASR pipeline,
right from pre-processing the audio inputs to decoding the
model predictions.

Running the example below will generate a Gradio demo where we
can record speech through the microphone of our computer and input it to
our fine-tuned Whisper model to transcribe the corresponding text:

In [ ]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="sanchit-gandhi/whisper-small-hi")  # change to "your-username/the-name-you-picked"

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

iface = gr.Interface(
    fn=transcribe,
    inputs=gr.Audio(source="microphone", type="filepath"),
    outputs="text",
    title="Whisper Small Hindi",
    description="Realtime demo for Hindi speech recognition using a fine-tuned Whisper small model.",
)

iface.launch()

## Closing Remarks

In this blog, we covered a step-by-step guide on fine-tuning Whisper for multilingual ASR
using 🤗 Datasets, Transformers and the Hugging Face Hub. For more details on the Whisper model, the Common Voice dataset and the theory behind fine-tuning, refere to the accompanying [blog post](https://huggingface.co/blog/fine-tune-whisper). If you're interested in fine-tuning other
Transformers models, both for English and multilingual ASR, be sure to check out the
examples scripts at [examples/pytorch/speech-recognition](https://github.com/huggingface/transformers/tree/main/examples/pytorch/speech-recognition).